In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import yfinance as yf
from scipy.signal import argrelextrema
# from collections import deque
# from matplotlib.lines import Line2D
import pandas_ta as ta
import ccxt
from datetime import datetime, timedelta
# import plotly.graph_objects as go
# import inspect 
import os
# import ccxt_data_fetch
from IPython.display import display, HTML
from tqdm import tqdm
import re

# pd.set_option('display.max_rows', None)
pd.reset_option("display.max_rows")

In [2]:
def calc():
    # data=ccxt_data() # using it will reset all the params to default parms
    # symbol='ETH/USDT'

    global order
    global peaks_df
    global troughs_df    

    if len(data) > 14:
    

        data['ta_rsi'] = ta.rsi(data['Close'], length = 14)

        coin_peak = argrelextrema(data['Close'].values, np.greater, order=order)[0]

        coin_peaks_df = data.iloc[coin_peak]['Close']
  

        ta_peak = argrelextrema(data['ta_rsi'].values, np.greater, order=order)[0]
        ta_peaks_df = data.iloc[ta_peak]['ta_rsi']
 

        coin_trough = argrelextrema(data['Close'].values, np.less, order=order)[0]
        coin_trough_df = data.iloc[coin_trough]['Close']

        ta_trough = argrelextrema(data['ta_rsi'].values, np.less, order=order)[0]
        ta_trough_df = data.iloc[ta_trough]['ta_rsi']

        peaks = [coin_peaks_df, ta_peaks_df]
        peaks_df = pd.concat(peaks, axis=1)
        peaks_df = peaks_df[(peaks_df.Close.notna()) & (peaks_df.ta_rsi.notna())]

        troughs = [coin_trough_df, ta_trough_df]
        troughs_df = pd.concat(troughs, axis=1)
        troughs_df = troughs_df[(troughs_df.Close.notna()) & (troughs_df.ta_rsi.notna())]

        if peaks_df.empty:
            # tof=False
            # print('empty peaks', symbol)
            return False

        elif troughs_df.empty:
            # print('empty troughs', symbol)
            return False
        
        else:
            return peaks_df, troughs_df, True

    else:
        # tof=False
        print('data length less than 14', symbol)
        return False




In [3]:

# peaks_df, troughs_df = calc()

def signal_dates(n=9):
    global sell
    global buy
    sell = []
    buy = []
    dates_of_peaks = []
    dates_of_troughs = []

    if calc():

        for i in range(1,n):

            diff = peaks_df.diff(i)
            # diff.dropna(inplace=True) # do not use it
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_peaks.append(sig.index.values)

    
            diff = troughs_df.diff(i)
            # diff.dropna(inplace=True) # do not use it
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_troughs.append(sig.index.values)

        for list in dates_of_peaks:
            for i in list:
                if str(i) not in sell:
                    # np.datetime to str
                    # maybe pd.to_datetime(i) works better
                    sell.append(str(i))

        for list in dates_of_troughs:
            for i in list:
                if str(i) not in buy:
                    buy.append(str(i))
                    

        sell.sort()
        buy.sort()

        # print(dates_of_peaks)
        # problem : buy ans sell should be list of datetime***********************

        # sell=pd.to_datetime(sell)
        # buy=pd.to_datetime(buy)


        if not buy or not sell:
            print('empty buy or sell')
            return False
        else:
            return sell, buy, True

    
            
    else:
        # tof=False
        # print(' calc() is false ', symbol)
        return False



In [4]:

n=9
order=5

timeframe='2h'
symbol = 'JASMYUSDT'

outdir=f"{os.getcwd()}/data/{timeframe}"
data = pd.read_csv(f"{outdir}/{symbol}_{timeframe}.csv", index_col='Time')


calc()
signal_dates(n)

# the real buy is on the open of the third candle
print(f'buy : {buy[-1]} \nsell : {sell[-1]}')



buy : 2022-03-16 06:00:00 
sell : 2022-03-17 02:00:00


In [5]:
# import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

for b in buy:
    fig.add_vline(x=b, line_width=1, line_dash="dash", line_color="green")
for s in sell:
    fig.add_vline(x=s, line_width=1, line_dash="dash", line_color="red")

               
fig.update_layout(template = 'plotly_dark')
fig.layout.xaxis.rangeslider.visible = False


fig.show()